<a href="https://colab.research.google.com/github/huongd17at089/detectAPT_2020/blob/main/Sqn2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
data_folder  = "/content/drive/My Drive/Graph2Vec/data/"
workers = 4
wl_iterations = 2
dimensions = 64
min_count = 1
down_sampling = 0.0001
epochs = 100
learning_rate = 0.01
output_path = "/content/drive/My Drive/Graph2Vec/vectors/vector.csv"
# saved model 
# "/content/drive/My Drive/Graph2Vec/model/model" 
seve_model_path = "/content/drive/My Drive/Graph2Vec/model/model1"

In [ ]:
def get_sequence(file_path):
  name = file_path.strip(".json").split("/")[-1]
  data = json.load(open(file_path, encoding="utf8"))
  sequence = []
  processes = data["Processes"]
  if processes:
      incidents = data["Incidents"]
      process_dict = {}
      for inc in incidents:
          if inc["MitreAttacks"] :
              process_dict[inc["ProcessOID"]] = process_dict.get(inc["ProcessOID"], []) + inc["MitreAttacks"]
      processes = sorted(processes, key=lambda d: d["CreationTimestamp"])
      for p in processes:
          sequence.append("-".join(process_dict.get(p["OID"], ["none"])))
  return TaggedDocument(words=sequence, tags=["s_" + name])
  


def save_to_csv(output_path, model, files, dimensions):
  out = []
  for f in files:
      identifier = f.split("/")[-1].strip(".json")
      vector = [identifier] + list(model.docvecs["s_"+identifier])
      if("malware" in identifier) :
          vector  = vector  + [1]
      else:
          vector  = vector  + [0]
      out.append(vector)
  column_names = ["id"]+["x_"+str(dim) for dim in range(dimensions)] + ["label"]
  out = pd.DataFrame(out, columns=column_names)
  out = out.sort_values(["id"])
  out.to_csv(output_path, index=None)

In [ ]:
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from joblib import Parallel, delayed
import glob
from gensim.test.utils import get_tmpfile
import os

files = []
for r, d, f in os.walk(data_folder):
  for file in f:
    if '.json' in file:
      files.append(os.path.join(r, file))

document_collections = Parallel(n_jobs=workers)(delayed(get_sequence)(f) for f in tqdm(files))

model = Doc2Vec(document_collections,
                    vector_size=dimensions,
                    window=2,
                    min_count=min_count,
                    dm=1,
                    sample=down_sampling,
                    workers=workers,
                    epochs=epochs,
                    alpha=learning_rate
                    )
# fname = get_tmpfile("test_model")
model.save(seve_model_path)

save_to_csv(output_path, model, files, dimensions)